WEEK 2: Deep Seek Store Assistant

Import Dependencies 

In [ ]:
import os
from dotenv import load_dotenv
import gradio as gr

Get Deepseek Key

In [ ]:


load_dotenv(override=True)
api_key = os.getenv('DEEPSEEK_API_KEY')
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
#elif not api_key.startswith("sk-proj-"):
#    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif not api_key.startswith("sk-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:


#Add Northwind Database
DB = "northwind.db"



System Message

In [ ]:
system_message = """
You are a helpful store assistant for Northwind Traders company.
Give short, courteous answers, no more than 1-2 sentences.
Always be accurate about product availability and pricing from our database.
If you don't know the answer or can't find the product, say so.
"""

Get product from the database

In [ ]:
def get_product(product_name):
    print(f"DATABASE TOOL CALLED: Getting product info for {product_name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT p.ProductID, p.ProductName, p.UnitPrice, p.UnitsInStock, c.CategoryName
            FROM Products p
            LEFT JOIN Categories c ON p.CategoryID = c.CategoryID
            WHERE p.ProductName LIKE ?''', (f'%{product_name}%',))
        result = cursor.fetchone()
        if result:
            product_id, name, price, in_stock, category = result
            category_info = f", Category: {category}" if category else ""
            return f"Product: {name} (ID: {product_id}), Price: ${price:.2f}, In Stock: {in_stock}{category_info}"
        else:
            return f"No product found with name containing '{product_name}'"

In [ ]:
get_product("Cheese")

Get Shop Assistant build tool

In [ ]:
from openai import OpenAI

# Initialize DeepSeek client
client = OpenAI(
    api_key=os.getenv('DEEPSEEK_API_KEY'),
    base_url="https://api.deepseek.com/v1"
)

def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
            temperature=0.7,
            max_tokens=500
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"I apologize, but I'm having trouble accessing our system right now. Please try again later."

gr.ChatInterface(fn=chat, type="messages").launch()